# Oracle LiveLabs - Forecasting service

[Oracle LiveLabs - Forecasting Service](https://apexapps.oracle.com/pls/apex/dbpm/r/livelabs/workshop-attendee-2?p210_workshop_id=927&p210_type=2&session=107363063846819)

In [ ]:
import pandas as pd
import requests
import json
import ast
import matplotlib.pyplot as plt
import re
import os
import simplejson

import oci

# you can also use resource principal.
signer = oci.auth.signers.InstancePrincipalsSecurityTokenSigner()
signer

## download sample data

```bash
# primary data
wget https://oracle.github.io/learning-library/oci-library/oci-hol/oci-artificial-intelligence/forecasting-service/item-4-forecasting-understand-data/files/favorita-13-beverages-primary.csv

# additional data
wget https://oracle.github.io/learning-library/oci-library/oci-hol/oci-artificial-intelligence/forecasting-service/item-4-forecasting-understand-data/files/favorita-13-beverages-add.csv
```

In [ ]:
# inline data preparation

# load data
df_primary = pd.read_csv('sample-data/favorita-13-beverages-primary.csv')
df_additional = pd.read_csv('sample-data/favorita-13-beverages-add.csv')

# modify data format to match forecasting service
df_primary['date'] = pd.to_datetime(df_primary['date'], format='%d/%m/%y').apply(lambda date: str(date))
df_additional['date'] = pd.to_datetime(df_additional['date'], format='%d/%m/%y').apply(lambda date: str(date))

# setting variables to create forecasting service
col_order_primary = ['date', 'sales', 'item_id']
col_order_additional = ['date', 'onpromotion', 'item_id']

primary_load = df_primary[col_order_primary].values.transpose().tolist()
additional_load = df_additional[col_order_additional].values.transpose().tolist()

primary_load
additional_load

In [ ]:
create_project_url = "https://forecasting.aiservice.us-phoenix-1.oci.oraclecloud.com/20220101/projects"

create_project_payload = json.dumps({
  "displayName": "Forecast via API",
  "compartmentId": os.environ['C'],
  "description": "Forecasting service API Demo",
  "freeformTags": None,
  "definedTags": None,
  "systemTags": None
})
create_project_headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", create_project_url, headers=create_project_headers, data=create_project_payload, auth=signer)

In [ ]:
create_project_response = json.loads(response.text)
create_project_response

# store the compartment_id and project_id
compartment_id=os.environ.get('C')
project_id=create_project_response['id']

print(compartment_id)
print(project_id)

In [ ]:
date_col_primary = 'date'
date_col_add = 'date'
target_col = 'sales'
id_col_primary = 'item_id'
id_col_add = 'item_id'
data_frequency = 'DAY'
forecast_frequency = 'DAY'
forecast_horizon = 14

In [ ]:
%%time

forecast_base_url = "https://forecasting.aiservice.us-phoenix-1.oci.oraclecloud.com/20220101/forecasts"

create_forecast_payload = json.dumps({
  "compartmentId": compartment_id,
  "displayName": "Forecast Model",
  "description": "Training Forecast Model",
  "projectId": project_id,
  "forecastCreationDetails": {
    "targetVariables": [
      target_col
    ],
    "modelTrainingDetails": {
      "modelType": "UNIVARIATE",
      "models": [
        "SMA",
        "DMA",
        "HWSA",
        "HWSM",
        "SES",
        "DES",
        "SA",
        "SM",
        "UAM",
        "UHM",
        "ARIMA",
        "PROPHET"
      ]
    },
    "forecastHorizon": forecast_horizon,
    "confidenceInterval": "CI_5_95",
    "errorMeasure": "RMSE",
    "forecastTechnique": "ROCV",
    "forecastFrequency": forecast_frequency,
    "isForecastExplanationRequired": True,
    "dataSourceDetails": {
      "type": "INLINE",
      "dataSources": {
        "primaryDataSource": {
          "isDataGrouped": True,
          "columnData": primary_load,
          "columnSchema": [
            {
              "columnName": date_col_primary,
              "dataType": "DATE"
            },
            {
              "columnName": target_col,
              "dataType": "INT"
            },
            {
              "columnName": id_col_primary,
              "dataType": "STRING"
            },
          ],
          "tsColName": date_col_primary,
          "tsColFormat": "yyyy-MM-dd HH:mm:ss",
          "dataFrequency": data_frequency
        },
        "additionalDataSource": {
          "isDataGrouped": True,
          "columnData": additional_load,
          "columnSchema": [
            {
              "columnName": date_col_add,
              "dataType": "DATE"
            },
            {
              "columnName": "onpromotion",
              "dataType": "INT"
            },
            {
              "columnName": id_col_add,
              "dataType": "STRING"
            },
          ],
          "tsColName": date_col_add,
          "tsColFormat": "yyyy-MM-dd HH:mm:ss",
          "dataFrequency": data_frequency
        }
      }
    }
  }
})

# print(forecast_payload)

create_forecast_headers = {
  "Content-Type": "application/json"
}

create_forecast_response = requests.request("POST", forecast_base_url, headers=create_forecast_headers, data=create_forecast_payload, auth=signer)

In [ ]:
create_forecast_response_json = json.loads(create_forecast_response.text)
create_forecast_response_json

In [ ]:
create_forecast_id = create_forecast_response_json['id']
create_forecast_id

In [ ]:
get_forecast_url = "https://forecasting.aiservice.us-phoenix-1.oci.oraclecloud.com/20220101/forecasts/{}".format(create_forecast_id)
print(get_forecast_url)

forecast_get_payload = {}
forecast_get_headers = {}

get_forecast_response = requests.request("GET", get_forecast_url, headers=forecast_get_headers,data = forecast_get_payload, auth=signer)

get_forecast_response_json = json.loads(get_forecast_response.text)
get_forecast_response_json

In [ ]:
df_forecasts = pd.DataFrame({'forecast_dates':[],'upper':[],'lower':[],'forecast':[], 'series_id':[]})
for i in range(len(get_forecast_response_json['forecastResult']['forecast'])):

    group = get_forecast_response_json['forecastResult']['forecast'][i]['targetColumn']
    point_forecast = get_forecast_response_json['forecastResult']['forecast'][i]['forecast']
    pred_intervals = pd.DataFrame(get_forecast_response_json['forecastResult']
                              ['forecast'][i]['predictionInterval'],dtype=float)
    out = pred_intervals
    out['forecast'] = point_forecast
    forecast_dates = pd.DataFrame({'forecast_dates':get_forecast_response_json['forecastResult']['forecast'][i]['dates']})
    forecasts = pd.concat([forecast_dates,out],axis=1)
    forecasts['series_id'] = group
    df_forecasts = df_forecasts.append(forecasts, ignore_index = False)
file_name = 'forecast.csv'
df_forecasts.to_csv(file_name, index = None)

In [ ]:
get_forecast_response_json['forecastResult']['metrics']['targetColumns']

In [ ]:
# get_forecast_description_url = "https://forecasting.aiservice.us-phoenix-1.oci.oraclecloud.com/20220101/forecasts/{}/explanations/".format(create_forecast_id)
get_forecast_description_url = "https://forecasting.aiservice.us-phoenix-1.oci.oraclecloud.com/20220101/forecasts/ocid1.aiforecast.oc1.phx.amaaaaaassl65iqaq5ohlekyqy5g6yaoiarth4nllx2xlysblafyumhyvzyq/explanations/"
print(get_forecast_description_url)

get_forecast_description_payload = {}
get_forecast_description_headers = {}

get_forecast_description_response = requests.request("GET", get_forecast_description_url, headers=get_forecast_description_headers, data=get_forecast_description_payload, auth=signer)
get_forecasting_description_json = json.loads(get_forecast_description_response.text)
get_forecasting_description_json

In [ ]:
%pip install plotly

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

df_description = pd.read_json('sample-data/Forecast_Explanation_ocid1.aiforecast.oc1.phx.amaaaaaassl65iqaq5ohlekyqy5g6yaoiarth4nllx2xlysblafyumhyvzyq.json')
global_feature_importance = df_description['explanations'][0]['global_feature_importance']['influencing_features']
# global_feature_importance = get_forecasting_description_json['explanations'][0]['globalFeatureImportance']['influencingFeatures']
df_imps = pd.DataFrame()
df_imps['Feature_Importance'] = global_feature_importance.values()

feature_names = global_feature_importance.keys()
df_imps['Features'] = feature_names

title = "Global Feature Importance "
fig = px.bar(df_imps, y="Features", x='Feature_Importance', 
            title=title).update_yaxes(categoryorder = "total ascending")
fig.update_traces(marker_color='lightsalmon')
fig.show()


In [ ]:
%pip install numpy

In [ ]:
import numpy as np

time_step=1

df_local_description = pd.read_json('./sample-data/Forecast_Explanation_ocid1.aiforecast.oc1.phx.amaaaaaassl65iqaq5ohlekyqy5g6yaoiarth4nllx2xlysblafyumhyvzyq.json')
local_feature_importance = df_local_description['explanations'][0]['local_feature_importance']['influencing_features'][time_step]
local_feature_importance
df_imps_local = pd.DataFrame()
df_imps_local['Feature_Importance'] = local_feature_importance.values()

feature_names = local_feature_importance.keys()
df_imps_local['Features'] = feature_names

title = "Local Feature Importance for Timestep " + str(time_step)
fig = px.bar(df_imps_local, y="Features", x='Feature_Importance', title=title)
fig.update_traces(marker_color='lightsalmon')
fig.show()